# 02 - Tipagem & Padronização – SRAG Hospitalizado

### Objetivo desta etapa

- Partir do `df_engenharia_sanitizado.csv` (tudo como string) e gerar um **df_tipado** consistente.
- **Tipar explicitamente** datas, numéricos, categorias e flags:
  - baseado em dicionário refinado + contexto epidemiológico + engenharia já feita.
- **Padronizar valores**:
  - strings (maiúsculas, espaços, rótulos),
  - missing (NaN),
  - categorias especiais (ignorados, missing, etc.).

### O que vamos fazer, em alto nível

- Definir tipos por **grupos de variáveis**:
  - **Datas** → colunas `_date` (`datetime64`), para atrasos/curvas.
  - **Numéricos de verdade** → `float` / `Int16` (ex.: idade).
  - **Binárias/flags** → `{0, 1, NaN}` em `Int8`/boolean.
  - **Categóricas codificadas** → `category` (`CS_RACA`, `EVOLUCAO`, etc.), usando `_label` / `_macro` na análise.
  - **Categóricas derivadas** → `category` (ou ordenada, no caso de gravidade).
  - **Geográficas** → `category` (`SG_UF`, município, zona).
  - **IDs/textos livres** → string
- Usar conversões **seguras**:
  - `pd.to_numeric(..., errors="coerce")`,
  - `pd.to_datetime(..., errors="coerce")`

- Manter **bruto + derivado**:
  - ex.: `EVOLUCAO` (código) + `EVOLUCAO_BIN` (0/1),
  - `CS_RACA`, `CS_RACA_label`, `CS_RACA_macro`.

### Saída esperada

- Um `df_tipado` salvo em `data/processed/` com:
  - datas em `datetime64`,
  - numéricos limpos,
  - flags binárias consistentes,
  - categorias em `category`,
  - e um mini “esquema” documentando as decisões de tipo.

> Esse `df_tipado` vira a **base oficial** para:
> - EDA de grupos de risco, gravidade, tempo, geografia,
> - Modelagem (óbito, UTI, gravidade),
> - Gráficos finais.


In [16]:
import pandas as pd
import numpy as np
from pathlib import Path
# Aumenta o número máximo de colunas mostradas pelo pandas.
# Útil para inspecionar bases largas sem que ele corte com "..."
pd.set_option("display.max_columns", 120)

# Caminhos Utilizados
PROJECT_ROOT = Path.cwd().parent        # sobe um nível

DATA_RAW = PROJECT_ROOT / "srag_2023_analysis" / "data" / "processed" 
CSV_PATH = DATA_RAW / "df_engenharia_sanitizado.csv"

In [23]:
df = pd.read_csv(CSV_PATH, dtype=str, low_memory=False)
print("Shape inicial:", df.shape)
df.head()

Shape inicial: (279453, 148)


,NU_NOTIFIC,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,CS_SEXO,DT_NASC,NU_IDADE_N,TP_IDADE,COD_IDADE,CS_GESTANT,CS_RACA,CS_ESCOL_N,ID_PAIS,CO_PAIS,SG_UF,ID_RG_RESI,CO_RG_RESI,ID_MN_RESI,CO_MUN_RES,CS_ZONA,NOSOCOMIAL,AVE_SUINO,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,OUTRO_DES,FATOR_RISC,PUERPERA,CARDIOPATI,HEMATOLOGI,SIND_DOWN,HEPATICA,ASMA,DIABETES,NEUROLOGIC,PNEUMOPATI,IMUNODEPRE,RENAL,OBESIDADE,OUT_MORBI,MORB_DESC,VACINA,MAE_VAC,M_AMAMENTA,ANTIVIRAL,TP_ANTIVIR,DT_ANTIVIR,HOSPITAL,...,DT_DIGITA,HISTO_VGM,PCR_SARS2,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,DT_TOMO,TP_TES_AN,DT_RES_AN,RES_AN,POS_AN_FLU,POS_AN_OUT,AN_SARS2,RES_IGG,RES_IGM,RES_IGA,POV_CT,TEM_CPF,ESTRANG,VACINA_COV,DOSE_1_COV,DOSE_2_COV,DOSE_REF,DOSE_2REF,DOS_RE_BI,FAB_COV_1,FAB_COV_2,FAB_COVRF,FAB_COVRF2,FAB_RE_BI,LOTE_1_COV,LOTE_2_COV,LOTE_REF,LOTE_REF2,LOT_RE_BI,FNT_IN_COV,TRAT_COV,SURTO_SG,CO_DETEC,REINF,CS_SEXO_label,CS_GESTANT_label,CS_RACA_label,CS_ESCOL_N_label,CS_ZONA_label,EVOLUCAO_label,UTI_label,SUPORT_VEN_label,CS_SEXO_clean,CS_RACA_macro,NU_IDADE_N_num,CS_ESCOL_N_label_qualidade,EVOLUCAO_BIN,EVOLUCAO_disponivel,UTI_flag,UTI_missing,UTI_ignorado,GRAVIDADE_RESPI
0,316004370301,2023-02-07,06,2023-02-05,06,SP,GVE VII SANTO ANDRE,1332,SAO CAETANO DO SUL,354880,F,1941-06-30,81,3,3081,6,1,1,BRASIL,1,SP,GVE VII SANTO ANDRE,1332,SAO CAETANO DO SUL,354880,1,2,2,NaN,NaN,NaN,1,NaN,NaN,NaN,1,1,RNC,1,NaN,1,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,1,ACAMADA HA 12 ANOS,2,NaN,NaN,2,NaN,NaN,1,...,2020-09-18,0,NaN,NaN,NaN,NaN,NaN,6,NaN,2,2023-02-07,2,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2,1,10/03/2021,07/04/2021,06/04/2022,06/04/2022,NaN,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,88 - COVID-19 JANSSEN - AD26.COV2.S,88 - COVID-19 JANSSEN - AD26.COV2.S,NaN,202010034,210083,210F21A,210F21A,NaN,2,NaN,NaN,NaN,2,F - Feminino,6 - Não se aplica,1 - Branca,1 - Fundamental 1º ciclo (1ª a,1 - Urbana,2 - Óbito,2 - Não,3 - Não,F,Branca,81,1 - Fundamental 1º ciclo (1ª a,1.0,1,0.0,0,0,0
1,316153805701,2023-02-28,09,2023-01-17,03,SP,GVE VII SANTO ANDRE,1332,SAO CAETANO DO SUL,354880,M,1933-12-10,89,3,3089,6,1,1,BRASIL,1,SP,GVE VII SANTO ANDRE,1332,SAO CAETANO DO SUL,354880,1,2,2,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,QUEDA DO ESTADO GERAL,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NEO PROSTATA/HIPOTIREOIDISMO,2,NaN,NaN,2,NaN,NaN,1,...,2021-03-10,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2023-01-17,2,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2,1,12/02/2021,05/04/2021,13/10/2021,NaN,NaN,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,NaN,NaN,202010034,210062,210448,NaN,NaN,2,NaN,NaN,NaN,2,M - Masculino,6 - Não se aplica,1 - Branca,1 - Fundamental 1º ciclo (1ª a,1 - Urbana,2 - Óbito,2 - Não,3 - Não,M,Branca,89,1 - Fundamental 1º ciclo (1ª a,1.0,1,0.0,0,0,0
2,316321420027,2023-05-02,18,2023-04-14,15,PR,02RS METROPOLITANA,1356,CURITIBA,410690,F,1943-06-04,78,3,3078,5,1,9,BRASIL,1,PR,02RS METROPOLITANA,1356,CURITIBA,410690,1,2,2,NaN,1,1,1,NaN,2,NaN,NaN,1,"CEFALEIA, ASTENIA",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,HIPERTENSAO ARTERIAL,2,NaN,NaN,2,NaN,NaN,1,...,2021-09-20,0,NaN,NaN,NaN,NaN,NaN,1,2021-09-16,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2,1,17/03/2021,07/04/2021,NaN,NaN,NaN,SINOVAC,NaN,NaN,NaN,NaN,210056,210129,NaN,NaN,NaN,1,NaN,2,2,2,F - Feminino,5 - Não,1 - Branca,9 - Ignorado,1 - Urbana,1 - Cura,2 - Não,"2 - Sim, não invasivo",F,Branca,78,9 - Ignorado,0.0,1,0.0,0,0,1
3,316384665515,2023-03-09,10,2023-03-05,10,SP,GVE VII SANTO ANDRE,1332,SAO CAETANO DO SUL,354880,F,1916-04-29,105,3,3105,6,1,1,BRASIL,1,SP,GVE VII SANTO ANDRE,1332,SAO CAETANO DO SUL,354880,1,2,2,NaN,1,NaN,1,1,1,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,1,...,2021-12-02,0,NaN,NaN,1,NaN,NaN,NaN,NaN,2,2023-03-07,2,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2,1,09/02/2021,05/03/2021,08/09/2021,NaN,NaN,86 

In [24]:
# 1. Padronização de strings e missing

# Troca strings vazias / espaços por NaN
df = df.replace({"": np.nan, " ": np.nan})

# strip em colunas object
for col in df.columns:
    if df[col].dtype == "object":
        df[col] = df[col].str.strip()

# opcional: tudo maiúsculo nas colunas "codigo/letra"
cols_upper = ["CS_SEXO", "CS_SEXO_clean", "UTI", "SUPORT_VEN",
              "NOSOCOMIAL", "AVE_SUINO", "SG_UF", "SG_UF_NOT"]
for col in cols_upper:
    if col in df.columns:
        df[col] = df[col].str.upper()

df.head()

,NU_NOTIFIC,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,CS_SEXO,DT_NASC,NU_IDADE_N,TP_IDADE,COD_IDADE,CS_GESTANT,CS_RACA,CS_ESCOL_N,ID_PAIS,CO_PAIS,SG_UF,ID_RG_RESI,CO_RG_RESI,ID_MN_RESI,CO_MUN_RES,CS_ZONA,NOSOCOMIAL,AVE_SUINO,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,OUTRO_DES,FATOR_RISC,PUERPERA,CARDIOPATI,HEMATOLOGI,SIND_DOWN,HEPATICA,ASMA,DIABETES,NEUROLOGIC,PNEUMOPATI,IMUNODEPRE,RENAL,OBESIDADE,OUT_MORBI,MORB_DESC,VACINA,MAE_VAC,M_AMAMENTA,ANTIVIRAL,TP_ANTIVIR,DT_ANTIVIR,HOSPITAL,...,DT_DIGITA,HISTO_VGM,PCR_SARS2,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,DT_TOMO,TP_TES_AN,DT_RES_AN,RES_AN,POS_AN_FLU,POS_AN_OUT,AN_SARS2,RES_IGG,RES_IGM,RES_IGA,POV_CT,TEM_CPF,ESTRANG,VACINA_COV,DOSE_1_COV,DOSE_2_COV,DOSE_REF,DOSE_2REF,DOS_RE_BI,FAB_COV_1,FAB_COV_2,FAB_COVRF,FAB_COVRF2,FAB_RE_BI,LOTE_1_COV,LOTE_2_COV,LOTE_REF,LOTE_REF2,LOT_RE_BI,FNT_IN_COV,TRAT_COV,SURTO_SG,CO_DETEC,REINF,CS_SEXO_label,CS_GESTANT_label,CS_RACA_label,CS_ESCOL_N_label,CS_ZONA_label,EVOLUCAO_label,UTI_label,SUPORT_VEN_label,CS_SEXO_clean,CS_RACA_macro,NU_IDADE_N_num,CS_ESCOL_N_label_qualidade,EVOLUCAO_BIN,EVOLUCAO_disponivel,UTI_flag,UTI_missing,UTI_ignorado,GRAVIDADE_RESPI
0,316004370301,2023-02-07,06,2023-02-05,06,SP,GVE VII SANTO ANDRE,1332,SAO CAETANO DO SUL,354880,F,1941-06-30,81,3,3081,6,1,1,BRASIL,1,SP,GVE VII SANTO ANDRE,1332,SAO CAETANO DO SUL,354880,1,2,2,NaN,NaN,NaN,1,NaN,NaN,NaN,1,1,RNC,1,NaN,1,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN,1,ACAMADA HA 12 ANOS,2,NaN,NaN,2,NaN,NaN,1,...,2020-09-18,0,NaN,NaN,NaN,NaN,NaN,6,NaN,2,2023-02-07,2,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2,1,10/03/2021,07/04/2021,06/04/2022,06/04/2022,NaN,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,88 - COVID-19 JANSSEN - AD26.COV2.S,88 - COVID-19 JANSSEN - AD26.COV2.S,NaN,202010034,210083,210F21A,210F21A,NaN,2,NaN,NaN,NaN,2,F - Feminino,6 - Não se aplica,1 - Branca,1 - Fundamental 1º ciclo (1ª a,1 - Urbana,2 - Óbito,2 - Não,3 - Não,F,Branca,81,1 - Fundamental 1º ciclo (1ª a,1.0,1,0.0,0,0,0
1,316153805701,2023-02-28,09,2023-01-17,03,SP,GVE VII SANTO ANDRE,1332,SAO CAETANO DO SUL,354880,M,1933-12-10,89,3,3089,6,1,1,BRASIL,1,SP,GVE VII SANTO ANDRE,1332,SAO CAETANO DO SUL,354880,1,2,2,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,1,QUEDA DO ESTADO GERAL,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NEO PROSTATA/HIPOTIREOIDISMO,2,NaN,NaN,2,NaN,NaN,1,...,2021-03-10,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2023-01-17,2,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2,1,12/02/2021,05/04/2021,13/10/2021,NaN,NaN,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,NaN,NaN,202010034,210062,210448,NaN,NaN,2,NaN,NaN,NaN,2,M - Masculino,6 - Não se aplica,1 - Branca,1 - Fundamental 1º ciclo (1ª a,1 - Urbana,2 - Óbito,2 - Não,3 - Não,M,Branca,89,1 - Fundamental 1º ciclo (1ª a,1.0,1,0.0,0,0,0
2,316321420027,2023-05-02,18,2023-04-14,15,PR,02RS METROPOLITANA,1356,CURITIBA,410690,F,1943-06-04,78,3,3078,5,1,9,BRASIL,1,PR,02RS METROPOLITANA,1356,CURITIBA,410690,1,2,2,NaN,1,1,1,NaN,2,NaN,NaN,1,"CEFALEIA, ASTENIA",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,HIPERTENSAO ARTERIAL,2,NaN,NaN,2,NaN,NaN,1,...,2021-09-20,0,NaN,NaN,NaN,NaN,NaN,1,2021-09-16,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2,1,17/03/2021,07/04/2021,NaN,NaN,NaN,SINOVAC,NaN,NaN,NaN,NaN,210056,210129,NaN,NaN,NaN,1,NaN,2,2,2,F - Feminino,5 - Não,1 - Branca,9 - Ignorado,1 - Urbana,1 - Cura,2 - Não,"2 - Sim, não invasivo",F,Branca,78,9 - Ignorado,0.0,1,0.0,0,0,1
3,316384665515,2023-03-09,10,2023-03-05,10,SP,GVE VII SANTO ANDRE,1332,SAO CAETANO DO SUL,354880,F,1916-04-29,105,3,3105,6,1,1,BRASIL,1,SP,GVE VII SANTO ANDRE,1332,SAO CAETANO DO SUL,354880,1,2,2,NaN,1,NaN,1,1,1,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,1,...,2021-12-02,0,NaN,NaN,1,NaN,NaN,NaN,NaN,2,2023-03-07,2,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2,1,09/02/2021,05/03/2021,08/09/2021,NaN,NaN,86 

In [25]:
# 2. Conversão de datas para datetime

DATE_COLS = [
    "DT_NOTIFIC",
    "DT_SIN_PRI",
    "DT_NASC",
    "DT_ANTIVIR",
    "DT_INTERNA",
    "DT_ENTUTI",
    "DT_SAIDUTI",
    "DT_RAIOX",
    "DT_COLETA",
    "DT_PCR",
    "DT_EVOLUCA",
    "DT_ENCERRA",
    "DT_DIGITA",
    "DT_TOMO",
    "DT_RES_AN",
]

date_newcols = {}

for col in DATE_COLS:
    if col in df.columns:
        new_col = col + "_date"
        date_newcols[new_col] = pd.to_datetime(df[col], errors="coerce")

# aqui ele insere todas as novas colunas de uma vez só
df = df.assign(**date_newcols).copy()

# checagem
df[[c for c in df.columns if c.endswith("_date")]].head()


,DT_NOTIFIC_date,DT_SIN_PRI_date,DT_NASC_date,DT_ANTIVIR_date,DT_INTERNA_date,DT_ENTUTI_date,DT_SAIDUTI_date,DT_RAIOX_date,DT_COLETA_date,DT_PCR_date,DT_EVOLUCA_date,DT_ENCERRA_date,DT_DIGITA_date,DT_TOMO_date,DT_RES_AN_date
0,2023-02-07,2023-02-05,1941-06-30,NaT,2023-02-07,NaT,NaT,NaT,2023-02-07,NaT,2023-02-08,2023-02-09,2020-09-18,NaT,2023-02-07
1,2023-02-28,2023-01-17,1933-12-10,NaT,2023-01-17,NaT,NaT,NaT,2023-01-17,NaT,2023-02-26,2023-02-28,2021-03-10,NaT,2023-01-17
2,2023-05-02,2023-04-14,1943-06-04,NaT,2023-05-12,NaT,NaT,2023-04-30,2023-05-02,2023-05-02,2023-05-15,2023-05-16,2021-09-20,2021-09-16,NaT
3,2023-03-09,2023-03-05,1916-04-29,NaT,2023-03-07,NaT,NaT,NaT,2023-03-07,2023-03-07,2023-03-07,2023-03-10,2021-12-02,NaT,2023-03-07
4,2023-01-21,2023-01-17,1927-01-05,NaT,2023-01-21,2023-01-24,NaT,2023-01-22,2023-01-17,2023-01-18,2023-01-28,2023-02-03,2022-01-26,NaT,NaT


In [26]:
# 3. Conversão de colunas numéricas principais

# idade numérica
if "NU_IDADE_N" in df.columns:
    df["NU_IDADE_N_num"] = pd.to_numeric(df["NU_IDADE_N"], errors="coerce").astype("Int16")

# semanas epidemiológicas (se quiser trabalhar numericamente)
for col in ["SEM_NOT", "SEM_PRI"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int16")

# flags que já existem como numérico em string: EVOLUCAO_BIN, GRAVIDADE_RESPI, etc.
bin_int_cols = [
    "EVOLUCAO_BIN",
    "EVOLUCAO_disponivel",
    "UTI_flag",
    "UTI_missing",
    "UTI_ignorado",
    "GRAVIDADE_RESPI",
]

for col in bin_int_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int8")

df[["NU_IDADE_N_num", "SEM_NOT", "SEM_PRI"]].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279453 entries, 0 to 279452
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   NU_IDADE_N_num  279453 non-null  Int16
 1   SEM_NOT         279453 non-null  Int16
 2   SEM_PRI         279453 non-null  Int16
dtypes: Int16(3)
memory usage: 2.4 MB


In [27]:
# 4. Comorbidades em formato 0/1 (1=Sim, 0=Não, NaN=Ignorado/ausente)

COMORB_COLS = [
    "PUERPERA",
    "CARDIOPATI",
    "HEMATOLOGI",
    "SIND_DOWN",
    "HEPATICA",
    "ASMA",
    "DIABETES",
    "NEUROLOGIC",
    "PNEUMOPATI",
    "IMUNODEPRE",
    "RENAL",
    "OBESIDADE",
    "OUT_MORBI",
]

flag_newcols = {}

for col in COMORB_COLS:
    if col not in df.columns:
        continue

    flag_newcols[col + "_flag"] = df[col].map({"1": 1, "2": 0}).astype("Int8")

# insere todas de uma vez e já copia pra desfragmentar
df = df.assign(**flag_newcols).copy()

df[[c for c in df.columns if c.endswith("_flag")]].head()

,UTI_flag,PUERPERA_flag,CARDIOPATI_flag,HEMATOLOGI_flag,SIND_DOWN_flag,HEPATICA_flag,ASMA_flag,DIABETES_flag,NEUROLOGIC_flag,PNEUMOPATI_flag,IMUNODEPRE_flag,RENAL_flag,OBESIDADE_flag,OUT_MORBI_flag
0,0,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,1
1,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
2,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1
3,0,<NA>,1,<NA>,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,1,<NA>,<NA>,<NA>,<NA>,<NA>


In [28]:
# 5. Garantir consistência nas variáveis de desfecho/gravidade

# EVOLUCAO_BIN já foi convertida p/ Int8 acima.
# Se por algum motivo não existir, recria:
if "EVOLUCAO" in df.columns and "EVOLUCAO_BIN" not in df.columns:
    df["EVOLUCAO_BIN"] = df["EVOLUCAO"].map(
        {"1": 0, "2": 1, "3": 1}
    ).astype("Int8")

# Se EVOLUCAO_disponivel não existir:
if "EVOLUCAO_disponivel" not in df.columns and "EVOLUCAO" in df.columns:
    df["EVOLUCAO_disponivel"] = df["EVOLUCAO"].isin(["1", "2", "3"]).astype("Int8")

# UTI_flag, UTI_missing, UTI_ignorado já foram tratados acima

# GRAVIDADE_RESPI já numeric; se não existir, podemos recriar depois com a lógica antiga
df[["EVOLUCAO", "EVOLUCAO_BIN", "UTI", "UTI_flag", "GRAVIDADE_RESPI"]].head()



,EVOLUCAO,EVOLUCAO_BIN,UTI,UTI_flag,GRAVIDADE_RESPI
0,2,1,2,0,0
1,2,1,2,0,0
2,1,0,2,0,1
3,2,1,2,0,0
4,2,1,1,1,3


In [29]:
# 6. Categóricas: labels e macros

# Colunas de rótulo que já existem
LABEL_COLS = [
    "CS_SEXO_label",
    "CS_GESTANT_label",
    "CS_RACA_label",
    "CS_ESCOL_N_label",
    "CS_ZONA_label",
    "EVOLUCAO_label",
    "UTI_label",
    "SUPORT_VEN_label",
    "CS_ESCOL_N_label_qualidade",
]

for col in LABEL_COLS:
    if col in df.columns:
        df[col] = df[col].astype("category")

# macro / agrupamentos
if "CS_RACA_macro" in df.columns:
    df["CS_RACA_macro"] = df["CS_RACA_macro"].astype("category")

# geográficas (UF de residência e de notificação)
for col in ["SG_UF", "SG_UF_NOT"]:
    if col in df.columns:
        df[col] = df[col].astype("category")

# códigos originais importantes como categoria
CODE_CAT_COLS = [
    "CS_SEXO",
    "CS_SEXO_clean",
    "CS_GESTANT",
    "CS_RACA",
    "CS_ESCOL_N",
    "CS_ZONA",
    "EVOLUCAO",
    "UTI",
    "SUPORT_VEN",
    "TP_IDADE",
    "NOSOCOMIAL",
]

for col in CODE_CAT_COLS:
    if col in df.columns:
        df[col] = df[col].astype("category")

df[LABEL_COLS + ["CS_RACA_macro"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279453 entries, 0 to 279452
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   CS_SEXO_label               279453 non-null  category
 1   CS_GESTANT_label            279453 non-null  category
 2   CS_RACA_label               279453 non-null  category
 3   CS_ESCOL_N_label            279453 non-null  category
 4   CS_ZONA_label               279453 non-null  category
 5   EVOLUCAO_label              279453 non-null  category
 6   UTI_label                   279453 non-null  category
 7   SUPORT_VEN_label            279453 non-null  category
 8   CS_ESCOL_N_label_qualidade  279453 non-null  category
 9   CS_RACA_macro               279453 non-null  category
dtypes: category(10)
memory usage: 2.7 MB


In [31]:
# 7. Visão geral dos tipos finais

df_tipado = df.copy()

df_tipado.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279453 entries, 0 to 279452
Columns: 176 entries, NU_NOTIFIC to OUT_MORBI_flag
dtypes: Int16(3), Int8(19), category(23), datetime64[ns](15), object(116)
memory usage: 1.6 GB
